In [1]:
import nltk
import re

from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

#nltk.download('wordnet')
#nltk.download('stopwords')

Import the articles we got from scraping

In [2]:
file = open("data/articles.txt", "r")
articles = dict()
contents = file.readlines()

In [3]:
for line in contents[1:]:
    title, content = line.split(";")
    articles[title] = content[1:-2] #Omit 0th and -1st characters as they are just unneeded quotation marks

In [4]:
print(len(articles.keys()))

1000


Function used to:
1. Remove annotations and "this article is a stub.." from the content
2. Tokenize and remove stopwords and isolated punctuation
3. Lemmatize the tokens using wordnet

In [5]:
def prepareArticle(content):
    #0.remove the annotations => [1], [2] etc., also remove "this article is a stub..."
    no_annotations = re.sub('\[\d\]', '', content)
    no_stubs = re.sub('This .* is a stub. You can help Wikipedia by expanding it.', '', no_annotations)
    lowercased = no_stubs.lower()
    #1.tokenize and remove stopwords and punctuation
    words = word_tokenize(lowercased)
    stop = stopwords.words("english")
    tokenized = []
    for word in words:
        if word in stop or word in ",./!?#%*()[]{}:\"\"\'\'\\-=+_``":
            continue
        else:
            tokenized.append(word)
    #2.lemmatize using wordnet
    wordnet = WordNetLemmatizer()
    lemmatized = []
    for word in tokenized:
        lemmatized.append(wordnet.lemmatize(word))
    return " ".join(lemmatized)

Some examples of it working

In [6]:
title = list(articles.keys())[111]
print(title)
print("=======================================")
print(articles[title])
print("=======================================")
print(prepareArticle(articles[title]))

Baron Beauchamp
   The titles Baron Beauchamp and Viscount Beauchamp have been created several times throughout English and British history. There is an extant Viscountcy of Beauchamp, held by the Seymour family, Marquesses of Hertford.  The name Beauchamp (French "beautiful/fair field"), Latinised to de Bello Campo ("from the beautiful field/fair field"), is borne by three of the most ancient Anglo-Norman families which settled in England during the Norman Conquest of 1066: Beauchamp of Worcestershire, of Somerset and of Bedfordshire.[5] The surname was taken from their respective manors in Normandy and there is no evidence of any shared origin between the families of that name seated in those three separate counties. The Bedfordshire branch died out in the male line after only two generations. The heir of the Somerset branch was the powerful Seymour family, whilst the Worcestershire branch achieved the greatest power and prominence as Earls of Warwick.  (Descendants of the feudal bar

Convert all articles

In [7]:
converted = dict()
for article in articles.keys():
    converted[article] = prepareArticle(articles[article])

In [8]:
print(len(converted.keys()))

1000


And save it in the articles.csv file

In [10]:
out = open("data/articles.csv", "w")
out.write("Title;Content\n")
for title in converted.keys():
    out.write(title+";"+converted[title]+"\n")